## Query
The most basic search that can be performed is the query.  The query is a free-formed text expression, similar to the type of query you can perform within Google, matching against documents within the specified view.

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.Data.Content, 1.0.0-beta4"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.Data.Analysis, 0.20.1 Refinitiv.Data.Content, 1.0.0-beta4

Loading extensions from `C:\Users\U8007876\.nuget\packages\microsoft.data.analysis\0.20.1\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.Data.Content.SearchService;
using Refinitiv.Data.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Data;
using System;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<DataTable>((dt, writer) =>
{
    if (dt.Columns.Count > 0)
    {
        var rows = new List<ICollection<IHtmlContent>>();

        // Process Each data row
        foreach(var row in dt.Rows.Cast<DataRow>())
        {
            var cells = new List<IHtmlContent>();
            foreach (var cell in row.ItemArray)
            {
                switch (cell)
                {
                    case null:
                        break;
                    case JValue val:
                        cells.Add(td(val.ToString()));
                        break;
                    case IEnumerable<JToken> list:
                        cells.Add(td($"[{string.Join(", ", list.Select(node => $"{node}"))}]"));
                        break;
                    default:
                        cells.Add(td(cell));
                        break;
                }                
            }
            rows.Add(cells);
        }
        
        // Header
        var headers = new List<IHtmlContent>();
        foreach (DataColumn col in dt.Columns)
            headers.Add(th(col.ColumnName) as IHtmlContent);

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
DesktopSession.Definition().GetSession().OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                                        .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})"))
                                        .Open();

2023-02-21 4:04:56 PM:DesktopSession. (State: DesktopSession is Pending)
2023-02-21 4:04:58 PM:DesktopSession. (Event: {
  "Contents": "Desktop Session Successfully Authenticated"
})
2023-02-21 4:04:58 PM:DesktopSession. (State: DesktopSession is Opened)


#### Query - Default behavior

In [5]:
// The default syntax for search is to specify a query expression.  By default, search will return a default set of 
// fields (Properties).
var response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                       .GetData();
response.Data.Table

ORGANISATION,"International Business Machines Corp, Public Company",,37036,
ORGANISATION,"Banco IBM SA, Private Company",,76208,
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NYSE Consolidated",55839165994,1097326,IBM
ORGANISATION,"Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company",,18062670,
QUOTExEQUITY,"Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex",21481052421,48924732,0#IBMF:
QUOTExEQUITY,"Euronext Amsterdam IBM Dividend Future Chain Contracts, Equity Future, USD, Euronext Amsterdam",21612423771,259118763,0#IBMDF:
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 1, Equity Future, USD, Eurex",21481052892,49450681,IBMFc1
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 2, Equity Future, USD, Eurex",21481053949,50092347,IBMFc2
QUOTExEQUITY,"Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1, Equity Future, USD, Euronext Amsterdam",21613372305,260213021,IBMDFc1
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 3, Equity Future, USD, Eurex",21481053950,50092348,IBMFc3


In [6]:
// By default, 10 documents will be returned.  Let's override that.
response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                   .Top(20)
                                                   .GetData();
response.Data.Table

ORGANISATION,"International Business Machines Corp, Public Company",,37036,
ORGANISATION,"Banco IBM SA, Private Company",,76208,
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NYSE Consolidated",55839165994,1097326,IBM
ORGANISATION,"Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company",,18062670,
QUOTExEQUITY,"Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex",21481052421,48924732,0#IBMF:
QUOTExEQUITY,"Euronext Amsterdam IBM Dividend Future Chain Contracts, Equity Future, USD, Euronext Amsterdam",21612423771,259118763,0#IBMDF:
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 1, Equity Future, USD, Eurex",21481052892,49450681,IBMFc1
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 2, Equity Future, USD, Eurex",21481053949,50092347,IBMFc2
QUOTExEQUITY,"Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1, Equity Future, USD, Euronext Amsterdam",21613372305,260213021,IBMDFc1
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 3, Equity Future, USD, Eurex",21481053950,50092348,IBMFc3
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 4, Equity Future, USD, Eurex",21481053951,50092349,IBMFc4


#### Query expressions
If we were to type in the following expression, we might come away believing the query will list all personnel within IBM that have a title
of CEO.  While, the results will indicate people related to IBM and people with a title of CEO, we cannot assume they are both.  That is, 
we will see results with the term IBM and CEO listed in the document.

In [7]:
response = Search.Definition(Search.View.SearchAll).Query("IBM CEO")
                                                   .GetData();
response.Data.Table

PERSON,"Arvind Krishna - International Business Machines Corp - Chairman of the Board, Chief Executive Officer",34417810693,34417810693
PERSON,"Rudy S. Karsan - Kenexa Corp - Chairman of the Board, Chief Executive Officer",34413248256,34413248256
PERSON,"Mohammed Farooq - Gravitant Inc - Chairman of the Board, Chief Executive Officer, Co-Founder",34415058863,34415058863
PERSON,"Simon J. Beaumont - IBM Credit LLC - Chairman of the Board, President, Chief Executive Officer",34421596181,34421596181
PERSON,"Dirk Buddensiek - Ibm Ix Berlin GmbH - Chairman of the Management Board, Chief Executive Officer, Founder",34421727104,34421727104
PERSON,"Deidre Paknad - Pss Systems Inc - President, CEO, Board Member",34414057544,34414057544
PERSON,Ron S Dembo - Algorithmics Inc - President & CEO,34414091235,34414091235
PERSON,Frank Chisholm - Aptsoft Corp - CEO & Founder,34414058841,34414058841
PERSON,Michael Knagenhjelm - Metamerge As(Ibm Corp) - Chief Executive Officer,34414139687,34414139687
PERSON,Lance Crosby - Softlayer Technologies Inc - Chief Executive Officer,34414372380,34414372380


What might give a more favorable result would be to query all documents related to people that contain the term '*IBM*'. From the result, I can visually see
titles and optionally filter out people that are not CEO's.  I can do this by selecting the '**People**' view.
In addition, I'm going to choose specific *child* elements of the **DocumentTitle** field.  As we see in our output from the above requests, the 
'**DocumentTitle**' is selected by default.  This property is a convenient collection of elements that nicely describes the document returned and is
made up of multiple sub-properties such as: '**DTSubjectName**', '**DTSimpleType**', '**DTSource**', and '**DTCharacteristics**'. 

By doing this, I can now see a nice
break-down of the titles (DTCharacteristics), name (DTSubjectName) and company (DTSimpleType).

In [8]:
var select = "DocumentTitle, DTSubjectName, DTSimpleType, DTSource, DTCharacteristics";

response = Search.Definition(Search.View.People).Query("IBM")
                                                .Select(select)
                                                .GetData();
response.Data.Table

"Arvind Krishna - International Business Machines Corp - Chairman of the Board, Chief Executive Officer",Arvind Krishna,International Business Machines Corp,"Chairman of the Board, Chief Executive Officer"
Gary D. Cohn - International Business Machines Corp - Vice Chairman,Gary D. Cohn,International Business Machines Corp,Vice Chairman
"James J. Kavanaugh - International Business Machines Corp - Senior Vice President, Chief Financial Officer",James J. Kavanaugh,International Business Machines Corp,"Senior Vice President, Chief Financial Officer"
"Nickle LaMoreaux - International Business Machines Corp - Chief Human Resource Officer, Senior Vice President",Nickle LaMoreaux,International Business Machines Corp,"Chief Human Resource Officer, Senior Vice President"
"Thomas W. Rosamilia - International Business Machines Corp - Senior Vice President - IBM Software, Chairman, North America",Thomas W. Rosamilia,International Business Machines Corp,"Senior Vice President - IBM Software, Chairman, North America"
Michelle H. Browdy - International Business Machines Corp - Senior Vice President and General Counsel,Michelle H. Browdy,International Business Machines Corp,Senior Vice President and General Counsel
"Nicolas A. Fehring - International Business Machines Corp - Vice President, Controller",Nicolas A. Fehring,International Business Machines Corp,"Vice President, Controller"
Robert F. Del Bene - International Business Machines Corp - General Manager - IBM Technology Lifecycle Services,Robert F. Del Bene,International Business Machines Corp,General Manager - IBM Technology Lifecycle Services
Alex Gorsky - International Business Machines Corp - Lead Independent Director,Alex Gorsky,International Business Machines Corp,Lead Independent Director
David N. Farr - International Business Machines Corp - Independent Director,David N. Farr,International Business Machines Corp,Independent Director


In [9]:
// Search for a specific ISIN using query
response = Search.Definition(Search.View.SearchAll).Query("US046353AQ14")
                                                   .GetData();
response.Data.Table

QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, MARKETAXESS AXESSALL",,0x00102cc599441a3e,US046353AQ14=MKAX
QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible, TRADEWEB DIRECT",,0x00102c8e989a2191,046353AQ1=TWBL
QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible, REAL TIME ALL QUOTES",,0x00102c4b5ce30dc5,0#046353AQ1=
QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, QUOTRIX",,0x00102c6132ee29a8,DEA19JW6=QTX
QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, STUTTGART STOCK EXCHANGE",,0x00102cc6f2eb0dee,DEA19JW6=SG
QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TRADEECHO APA EU",,0x00102cec7f8b2f0b,US046353AQ14=TRU
QUOTExFIXEDINCOMExGOVCORP,"NT, Note, US046353AQ14 MiFID II Fine Grained RIC, Marketaxess Europe Limited",196761311753,81618214828,US046353AQ14=MAXM
QUOTExFIXEDINCOMExGOVCORP,"NT, Note, US046353AQ14 MiFID II Fine Grained RIC, Deutsche Boerse AG APA Service",196764266323,81620452767,US046353AQ14=DAp
INSTRUMENTxFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",46645702869,0x00102cb50fde0d55,


In [10]:
// Narrow the search to only instruments
response = Search.Definition(Search.View.Instruments).Query("US046353AQ14")
                                                     .GetData();
response.Data.Table

INSTRUMENTxFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",46645702869,0x00102cb50fde0d55


#### Query Syntax
A query phrase can support some basic syntax including wildcards, boolean expressions and exact phrase matching.

In [11]:
// Use a specifc term in quotes and the boolean operator to locate specific tankers - use the Content layer 
// to print additional details.
response = Search.Definition(Search.View.SearchAll).Query("'Caribbean Tanker' and Baltic")
                                                   .GetData();
Console.WriteLine($"Matched a total of {response.Data.Total} documents");
response.Data.Table

Matched a total of 23 documents


QUOTExCOMMODITY,"Baltic Exchange TD9 CRB-USG, Commodity Index, The Baltic Exchange",21815646948,621981,.BACG
QUOTExCOMMODITY,"Baltic Exchange TC 21 US Gulf to Caribbean, Commodity Spot, The Baltic Exchange",21937072473,661825555,.BAGCR
QUOTExCOMMODITY,"Baltic Exchange Dirty Tanker TD21 Caribbean to US Gulf (50,000mt fuel oil), Commodity Spot, The Baltic Exchange",21717867959,175653765,.BACR
QUOTExCOMMODITY,"Baltic Exchange Dirty Tanker TD9 TCE Caribbean to US Gulf Time Charter, Commodity Spot, The Baltic Exchange",21717782063,48950908,.BAAFRTD9
QUOTExCOMMODITY,"Baltic Exchange Dirty Tanker TD21 TCE Caribbean to US Gulf (50,000mt fuel oil) Time Charter, Commodity Spot, The Baltic Exchange",21717782060,325658314,.BAAFRTD21
PHYSICALASSETxVESSEL,"BEKS OCEAN, Panamax, BEKS DENIZCILIK VE TICARET, English Channel|UK Continent Tanker Zone",77309486082,77309486082,C}KP7309486082
PHYSICALASSETxVESSEL,"MANDO ONE, Medium, TSAKOS COLUMBIA SHIPMANAGEMENT, Caribbean Sea|Bahamas Tanker Zone",77309486968,77309486968,C}KM7309486968
PHYSICALASSETxVESSEL,"NAVE PULSAR, Medium, NAVIOS TANKERS MANAGEMENT INC, Caribbean Sea|Caribbean Tanker Zone|Puerto Rico",77309494434,77309494434,C}KM7309494434
PHYSICALASSETxVESSEL,"TORM LILLY, Medium, TORM A/S, Baltic Sea|Baltic Tanker Zone",77309495565,77309495565,C}KM7309495565
PHYSICALASSETxVESSEL,"TAURUS, Medium, KONDINAVE SA, Leningrad|Gulf of Finland|Baltic Tanker Zone|Russia",77309497715,77309497715,C}KM7309497715


In [12]:
// Use a wildcard (*) to search for references to 'Covid*'.
select = "DTSubjectName, DTSimpleType, DTSource, DTCharacteristics";

response = Search.Definition(Search.View.SearchAll).Query("Covid*")
                                                   .Select(select)
                                                   .GetData();
Console.WriteLine($"Matched a total of {response.Data.Total} documents");
response.Data.Table

Matched a total of 8556 documents


Covidh Technologies Ltd,Public Company,,
Mass General Brigham Inc,Private Company,,
Covidh Technologies Ltd,Ordinary Share,BSE Ltd,
Covidien International Finance SA,Private Company,,
Banconal COVID Relief Facility SARL,Private Company,,
COVF Spread Fut.,Equity Future,Eurex,USD
Covidien Ventures,Private Company,,
"United States, Coronavirus (COVID-19), Vaccinations, People vaccinated, Volume",Population,Our World in Data,Daily
"United States, Coronavirus (COVID-19), Vaccinations, People vaccinated, % of total",Population,Our World in Data,Daily
"United States, Coronavirus (COVID-19), Vaccinations, People fully vaccinated, Volume",Population,Our World in Data,Daily


In [13]:
// List the Vodafone bonds maturing in 2025 with a coupon rate of 5%
// We can achieve this specific query using a special feature called 'tcm' (ticker coupon maturity).
select = "CouponCurrency, CouponTypeDescription, IssuerLegalName, CouponRate, MaturityDate, ISIN";

response = Search.Definition(Search.View.SearchAll).Query("vod 5% 2025")
                                                   .Features("tcm")
                                                   .Select(select)
                                                   .GetData();
response.Data.Table

USD,Fixed:Plain Vanilla Fixed Coupon,Vodafone Hessen GmbH & Co. KG,5,2025-01-15 12:00:00 AM,USD85455AD13
USD,Fixed:Plain Vanilla Fixed Coupon,Vodafone Hessen GmbH & Co. KG,5,2025-01-15 12:00:00 AM,US913364AD74
